In [4]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from random import randint
from time import time

In [34]:
url = 'https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&sort=num_votes,desc&start=51&ref_=adv_nxt'

response = get(url)
print(response.text[:300])




<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
         
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imd


In [35]:
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)
movie_containers = html_soup.find_all('div', class_ = 'lister-item mode-advanced')

print(type(movie_containers))
print(len(movie_containers))

<class 'bs4.element.ResultSet'>
50


In [36]:
first_movie = movie_containers[0]#movie message
print ("電影名稱 : ",first_movie.h3.a.text)

電影名稱 :  奇蹟男孩


In [37]:
first_year = first_movie.h3.find('span', class_ = 'lister-item-year text-muted unbold')
first_year = first_year.text[1:5]
print ("發行年份 : " , first_year)

發行年份 :  I) (


In [38]:
first_imdb = float(first_movie.strong.text)
print ("IMDB評級 : " , first_imdb)

IMDB評級 :  8.0


In [39]:
first_mscore = first_movie.find('span', class_ = 'metascore favorable')
first_mscore = int(first_mscore.text)
print("Metascore 評分 :" , first_mscore)

Metascore 評分 : 66


In [40]:
first_votes = first_movie.find('span', attrs = {'name':'nv'})
print (first_votes.text)
print ("投票數 : " +first_votes['data-value'])#int

125,271
投票數 : 125271


In [41]:
names = []
years = []
imdb_ratings = []
metascores = []
votes = []

# Extract data from individual movie container
for container in movie_containers:

    # 僅當容器具有Metascore時才提取感興趣的數據點:
    if container.find('div', class_ = 'ratings-metascore') is not None:

        # The name
        name = container.h3.a.text
        names.append(name)

        # The year
        year = container.h3.find('span', class_ = 'lister-item-year').text
        years.append(year)

        # The IMDB rating
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)

        # The Metascore
        m_score = container.find('span', class_ = 'metascore').text
        metascores.append(int(m_score))

        # The number of votes
        vote = container.find('span', attrs = {'name':'nv'})['data-value']
        votes.append(int(vote))
test_df = pd.DataFrame({'movie': names,
                       'year': years,
                       'imdb': imdb_ratings,
                       'metascore': metascores,
                       'votes': votes})
print(test_df.info())
# 從最後開始計算，我們可以看到年份總是從第五個字符到第二個字符。我們將使用該.str() 方法僅選擇該間隔。
# 我們還將使用以下astype() 方法將結果轉換為整數：


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   movie      37 non-null     object 
 1   year       37 non-null     object 
 2   imdb       37 non-null     float64
 3   metascore  37 non-null     int64  
 4   votes      37 non-null     int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.6+ KB
None


In [42]:
test_df

,movie,year,imdb,metascore,votes
0,奇蹟男孩,(I) (2017),8.0,66,125271
1,大災難家,(2017),7.4,76,124888
2,樂高蝙蝠俠電影,(2017),7.3,75,124318
3,郵報：密戰,(2017),7.2,83,121370
4,決勝女王,(2017),7.4,71,119963
5,黑塔,(2017),5.6,34,118972
6,羅根好好運,(2017),7.0,78,116872
7,愛情昏迷中,(2017),7.5,86,112296
8,猜火車2,(2017),7.2,67,104937
9,忌日快樂,(2017),6.5,58,103917


In [1]:
import requests
from bs4 import BeautifulSoup


def main():
    print('蘋果今日焦點')
    dom = requests.get('http://www.appledaily.com.tw/appledaily/hotdaily/headline').text
    soup = BeautifulSoup(dom, 'html.parser')
    for ele in soup.find('ul', 'all').find_all('li'):
        print(
            ele.find('div', 'aht_title_num').text,
            ele.find('div', 'aht_title').text
        )
    print('-----------')


if __name__ == '__main__':
    main()

蘋果今日焦點
01 辛龍不再娶拒送山頭 追思會避疫取消
02 疫情顯微鏡：今年感染少 明年恐受創大（蘇益仁）
03 辛龍和霓霓過雙雙節續愛 欣慰劉真頭七入夢
04 劉真老公預約雙塔位 化煙飛佇真龍殿
05 籌辦舞蹈追思會 李志堯7月跳給劉真看
06 楊麗菁放不下閨蜜 楊千霈影片告別
07 高風險區回台 擬全面採檢 歐美返國確診暴增 學者建議社區設監測站快篩
08 蹺班接確診兒 主祕遭拔官 兒被查出在菲住院3天 返台填寫無症狀 竟1傳2
09 單日3命 衝破300例 實踐師發病24天確診 台增至5死
10 【台灣關鍵14天 Day12】單挑WHO 外長筆戰抗中國打壓
11 開聘金1.4億求愛 34歲女主播不嫁
12 強人干政 引爆政爭 郝柏村 101歲辭世
13 每天只吃一碗飯 勤運動養生
14 人渣文本專欄：自私也該有個限度（周偉航）
15 喜劇泰斗 確診6天驟逝 70歲志村健謝幕
16 王品買1股就有 爽領2200元禮券
17 德國議員提案 挺台參與WHA
18 蘋中信：一場疫病，劃出界線（陳芳明）
19 涉掏空遠航22億 張綱維聲押
20 【Video Talk】志村健慢性病+癮君子 吳昌騰：70歲死亡率8%
21 平均4分鐘1死 紐約淪鬼城 中央公園建野戰醫院 電視畫面滿地屍袋嚇壞川普
22 口罩實名制升級 4/9起14天買9片
23 阿喜夜半曬豪乳 32D色誘釀血災
24 搭狗狗猩猩暴紅 思念小龐大哭
25 李愛綺泰國睡紅燈區 偽人妖惹同行眼紅
26 黃仲崑仙境練功 阻病毒超Man
27 掐脖狠K頭 名校師霸凌生 家長提告 師遭判拘100天
28 愛台灣訪逾10次 與金城武拍廣告
29 黃子佼延1年擺酒 完美婚紗沖喜疫情
30 裕隆慘賠244億 近24年來首虧 去年每股淨損15.55元 決議減資36%
-----------
